<a href="https://colab.research.google.com/github/datduyng/cs411-data-modeling/blob/master/hw03/Assignment03Part_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
from time import time
import os
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from matplotlib import pyplot as plt
from google.colab import files
import io
from tqdm import tqdm as loader


%matplotlib inline
plt.rcParams['figure.figsize'] = (20,15) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.style.use('ggplot')


# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving movies_metadata.csv to movies_metadata.csv


In [ ]:
movies= pd.read_csv(io.BytesIO(uploaded['movies_metadata.csv']))

In [ ]:
# Extra Credit 

def euclidean(list_A, list_B):
  diff_vect = list_A - list_B
  return np.sqrt(np.dot(diff_vect,diff_vect))


def manhattan(list_A, list_B):
  # list_length_check(list_A,list_B)
  from numpy import sqrt
  a_b = zip(list_A, list_B)
  manhattan = 0
  for x,y in a_b:
      manhattan += abs(x-y)
  return manhattan

def Distance_matrix(X, distance_metric):
  manhat = np.zeros(shape=(len(X),len(X)))
  euclid = np.zeros(shape=(len(X),len(X)))
  for i in range(len(X)):
    for j in range(len(X)):
      if euclid[i][j] == 0 & manhat[i][j] == 0 & i!=j:
        if distance_metric is 'euclidean':
          euclid[i][j] = euclidean(X[i],X[j])
          euclid[j][i] = euclid[i][j]
        if distance_metric is 'manhattan':
          manhat[i][j] = manhattan(X[i],X[j])
          manhat[j][i] = manhat[i][j]
  return euclid, manhat

  def RangeQueryX(X, distance_metric, euclid, manhat, point, eps ):
    N=[]
    idx=[]
    dist = 100
    if distance_metric is 'euclidean':
      idx = (euclid[point]<= eps).nonzero()[0]
    if distance_metric is 'manhattan':
      idx=(manhat[point]<= eps).nonzero()[0]
    N.append(idx)
    return N

In [ ]:
def euclidean(list_A, list_B):
  diff_vect = list_A - list_B
  return np.sqrt(np.dot(diff_vect,diff_vect))
  # return distance.euclidean(list_A, list_B)


def manhattan(list_A, list_B):
  # list_length_check(list_A,list_B)
  from numpy import sqrt
  a_b = zip(list_A, list_B)
  manhattan = 0
  for x,y in a_b:
      manhattan += abs(x-y)
  return manhattan

# def RangeQuery(X,distsnce_metric,point,eps):
#   N = distance.cityblock(X[i], point, w=None)
#   return N

def RangeQuery(X, distance_metric, point, eps):
  N = []
  dist = 100
  for i in (range(len(X))):
      if distance_metric is 'euclidean':
        dist = euclidean(X[i],point)
      if distance_metric is 'manhattan':
        dist = manhattan(X[i],point)
      if dist <= eps:
        N.append(i)
  return N

In [ ]:
class DBSCAN():
    def fit(self, X, eps, min_samples, distance_metric):
        C=0
        label = dict.fromkeys(range(len(X)), "unvisited")
        for i in loader(range(len(X))):
          if (label[i] == "unvisited"):
            N = RangeQuery(X, distance_metric, X[i], eps)
            if (len(N)<min_samples):
              label[i] = -1
            C = C+1
            label[i]= C
            N.remove(i)
            S = N
            for Q in range(len(S)):
                if label[S[Q]] == -1:
                  label[S[Q]] = C
                if label[S[Q]] == "unvisited":
                  label[S[Q]] = C
                Nh = RangeQuery(X, distance_metric, X[Q], eps)
                if len(Nh) >= min_samples:
                    S.append(Nh)
        return label

    def __init__(self):
            pass
            

In [ ]:

#retain numeric columns for clustering
movies = movies[['budget','popularity','revenue','runtime','vote_average','vote_count']]

# drop rows with null or NaN values
movies = movies[~movies.isnull().any(axis=1)]
movies = movies.dropna()

# remove movies with less than 30 votes
movies = movies[movies['vote_count'] > 30]
movies.head()

In [ ]:
scaler = StandardScaler()
movies_standardized = scaler.fit_transform(movies.values)

In [ ]:
movies.columns

In [ ]:
{'eps': 10.0,
 'min_samples': 4,
 'distance_metric': 'euclidean',
 'num_labels': 2,
 'silhouette': 0.9428480074887184}

### Find Optimal EPS

In [ ]:
#findOptimal EPS
dist = np.zeros(shape=(len(movies_standardized),len(movies_standardized)))
for i in loader(range(len(movies_standardized))):
  for j in range(len(movies_standardized)):
    dist[i][j] = euclidean(movies_standardized[i],movies_standardized[j])

# distdf=pd.DataFrame(np.sort(dist.values, axis=0), index=dist.index, columns=dist.columns)


In [ ]:
dist_sorted = np.sort(dist, axis=None)  

In [ ]:
plt.plot(dist_sorted)
plt.ylabel(' Distances')
plt.show()

### Run DBSCAN 

In [ ]:
eps = [0.1, 1.0, 2.0, 3.5, 5.0, 10.0]
min_samples = [4, 9, 15, 30, 50]
distance_metric = ['euclidean', 'manhattan']

dbscan_basket = []
best_param, best_score, best_labels = None, -1., []

In [ ]:
for e in eps:
    for ms in min_samples:
        for dm in distance_metric:
            dbscan = DBSCAN()
            labels = dbscan.fit(movies_standardized,e,ms,dm)
            num_labels = np.unique(labels).shape[0]
            if num_labels < 2: continue
            sil = silhouette_score(movies_standardized, labels)
            params = {
                'eps': e,
                'min_samples': ms,
                'distance_metric': dm,
                'num_labels': num_labels,
                'silhouette': sil
            }

            dbscan_basket.append(params)
            if sil > best_score:
                best_param = params
                best_score = sil
                best_labels = labels

In [ ]:
dbscan = DBSCAN()
labels = dbscan.fit(movies_standardized, 5,1,'euclidean')


100%|██████████| 12178/12178 [18:44<00:00, 10.83it/s]


In [ ]:
labels

In [ ]:
test = RangeQuery(movies_standardized, 'manhattan', movies_standardized[2], 2)

In [ ]:
best_param

In [ ]:
euclid,manh = Distance_matrix(movies_standardized, 'euclidean')

KeyboardInterrupt: ignored

In [ ]:
euclid

In [ ]:
manh

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])